# Day 3: Knowledge-Based Robotic Decision-Making
Welcome to the third day of our hands-on course!
Today, you will focus on understanding how a knowledge base supports robotic decision-making. You’ll learn to query the knowledge base to identify necessary actions for tasks, like how to perceive the milk inside the fridge.
### Goal
By the end of the session, you will have successfully made queries to the knowledge base, enabling the robot to determine the steps required to complete its tasks.


## Theoretical Background
ToDo: We will provide an overview of how knowledge bases operate in the context of robotics. You’ll learn how the robot can understand the need to open the fridge door to perceive the milk inside.


## Step-by-Step Hands-On Exercises

Import the necessary modules and define the objects in the environment, as we did in Day 2.

In [ ]:
from pycram.ros.tf_broadcaster import TFBroadcaster
from pycram.ros.viz_marker_publisher import VizMarkerPublisher, AxisMarkerPublisher
from pycram.worlds.bullet_world import BulletWorld
from pycram.designators.action_designator import *
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *
from pycram.datastructures.enums import ObjectType, WorldMode, TorsoState
from pycram.datastructures.pose import Pose
from pycram.process_module import simulated_robot, with_simulated_robot
from pycram.object_descriptors.urdf import ObjectDescription
from pycram.world_concepts.world_object import Object
from pycram.datastructures.dataclasses import Color
from pycram.designators.object_designator import BelieveObject
from pycram.designators.designator_notation_interface import ActionDesignator as Action
from pycram.external_interfaces.rosprolog_interface import Prolog as knowrob_client



### Query the Knowledge Base

Before we can start quering our knowledge base, we have to establish a connection to it by creating an object for the client:

In [ ]:
knowrob = knowrob_client()

If this has run through successfully (you should see something like "[KnowRbo] done"), we can test the connection by sending a test query written in Prolog:

In [ ]:
knowrob.once("member(X, [1,2,3]).")

Within the PyCram knowrob interface, we differenciate between wanting to receive just one result, or all possible solutions. For this, there are essentially two query functions: **once** and **all_solutions**. In most instances, it is fine to just use **once**. However **all_solutions** might be useful, for building failure handling functionality, in order to iterate over all the possible solutions. To see the difference for the member function, try it out with **all_solutions** instead of **once**. 
Do you notice any other difference between the two functions?
<details>

<summary>Click here to get the solution</summary>

```python
knowrob.all_solutions('member(X,[1,2,3]).')
```

The result of **once** is a dictionary, while the result of **all_solutions** is a list of dict. Keep this in mind when working on the results of KnowRob. 
</details>

Now, we would like to go back to our scenario of getting the milk from the fridge. Since the robot doesn't yet know where the milk is located, we could see if maybe knowrob does by posting a query. 
**Important** note: KnowRob uses Prolog as a query language, which allows it to be very powerful, but takes a while to get used to. Think of it as trying to express what you would like to know in a logic formula, and prolog will try and find a solution which matches your query.

The most important thing to know about it is, that everything starting with a capital letter is a variabe which Prolog will fill with knowledge which matches the query, and that you can chain queries with a comma, since that is a logical "and".

<details>

<summary><b> Prolog cheat sheet </b> </summary>


| Symbol          | Meaning                                | Example                                               |
|-----------------|----------------------------------------|-------------------------------------------------------|
| `.`             | End of clause or query                 | `likes(john, pizza).`                                 |
| `,`             | Logical AND                            | `happy(X), healthy(X).`                               |
| `;`             | Logical OR                             | `happy(X); sad(X).`                                   |
| `:-`            | "If" (defines a rule)                  | `happy(X) :- enjoys(X, Y), positive(Y).`              |
| `?-`            | Start a query                          | `?- likes(john, pizza).`                              |
| `=`             | Unification (binds values)             | `X = john.`                                           |
| `is`            | Arithmetic assignment                  | `X is 3 + 4.`                                         |
| `<`, `>`, `=<`, `>=` | Comparison operators        | `X > 5, Y =< 10.`                                     |
| `\=`            | Not equal                              | `X \= john.`                                          |
| `[]`            | Empty list                             | `X = [].`                                             |
| `\|`            | List cons (head and tail separator)    | `[H \| T] = [1, 2, 3].`                               |
| `_`             | Anonymous variable (ignored)           | `likes(_, pizza).`                                    |
| `\+`            | Negation                               | `\+ happy(X).`                                        |
| `!`             | Cut (prevents backtracking)            | `happy(X) :- enjoys(X, Y), !, positive(Y).`           |


</details>

In [ ]:
knowrob.once('entity (an Object(type="Milk", storagePlace="?storagePlace"))')

With this query we say that we are looking for an **entity**, which is an **Object**, which corresponds to the PyCRAM Object Designators. This object should be of **type** milk, and with **?storagePlace** we designate that we would like to know the **storage place** of the object, hence the questionmark. You can think of everything prefixed with a questionmark, as a varibale which gets filled with information by knowrob. 

In [ ]:
# Example code to query the knowledge base
# query which finds out if the fridge is open or not
query = 'entity ( an Object (type="Fridge",location= an Location(handle="?handle" openingState="?open")))'
# Run the query on the knowledge base
result = knowrob.once(query)
print(result)

### Plan Execution

In [ ]:
extension = ObjectDescription.get_file_extension()
world = BulletWorld(WorldMode.DIRECT)
viz = VizMarkerPublisher()
tf = TFBroadcaster()

pr2 = Object("pr2", ObjectType.ROBOT, "pr2.urdf")
apartment = Object('apartment', ObjectType.ENVIRONMENT, f'apartment{extension}')
# milk = Object("milk", ObjectType.MILK, "milk.stl", pose=Pose([0.7,2.5, 0.9]))
#milk.color = Color(0, 0, 1, 1)
milk_desig = BelieveObject(names=["milk"])
robot_desig = BelieveObject(names=["pr2"])
apartment_desig = BelieveObject(names=["apartment"])

## Opening Action
Opening allows the robot to open a Container, the container is identified by an ObjectPart designator which describes the handle of the drawer that should be grasped. The OpeningAction needs to know which arm should be used to open the container. The ObjectPart would look like this:  

```python
ObjectPart(names=["frdige_door_link_handle"], part_of=apartment_desig)
```  

It takes the name of the handle as a string and the part_of designator of the apartment. This name is corresponding to the name of the handle in the URDF file. As you can tell naming all the parts in the URDF file is crucial for the robot to be able to interact with them, but knowing them during coding is quite annoying and frustrating. But we can use our knowledge base to ask for specific parts or names of objects.  


In [ ]:
# Placeholder code for querying the knowledge base for the name of the fridge door
query = 'entity (an Object(type="Fridge",location= an Location(handle="?handle" openingState="?open")))'
actions = knowrob.once(query)
for action in actions:
    print(action)

Another side note, how does the robot now actual knows that it has to open fridge, obviously we have to tell it. But how? We can use the knowledge base to ask fo required actions. 

In [ ]:
# Placeholder code for querying the knowledge base for the name of the fridge door
query = 'required_action(robot, perceive, milk).'
actions = knowrob.once(query)
for action in actions:
    print(action)

now having the action we can use the OpeningAction to open the fridge door. so move the robot to the fridge door and open it!


In [ ]:
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    NavigateAction([Pose([0, 1, 0], [0, 0, 0, 1])]).resolve().perform()
    handle_desig = ObjectPart(names=["handle_cab3_door_top"], part_of=apartment_desig.resolve())
    drawer_open_loc = AccessingLocation(handle_desig=handle_desig.resolve(),
                                        robot_desig=robot_desig.resolve()).resolve()
    NavigateAction([drawer_open_loc.pose]).resolve().perform()
    OpenAction(object_designator_description=handle_desig, arms=[Arms.RIGHT]).resolve().perform()
   

<details>

<summary>Click here to get the solution</summary>

```python

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()
    NavigateAction([Pose([0, 1, 0], [0, 0, 0, 1])]).resolve().perform(
    handle_desig = ObjectPart(names=["handle_cab3_door_top"], part_of=apartment_desig.resolve())
    drawer_open_loc = AccessingLocation(handle_desig=handle_desig.resolve(),
                                        robot_desig=robot_desig.resolve()).resolve()
    NavigateAction([drawer_open_loc.pose]).resolve().perform()
    OpenAction(object_designator_description=handle_desig, arms=[Arms.RIGHT]).resolve().perform()
```
</details>

So what is missing look detect the milk and grasp it.


In [ ]:
with simulated_robot:
    

  
<details>

<summary>Click here to get the solution</summary>

```python

with simulated_robot:
      LookAtAction(targets=[milk_desig.resolve().pose]).resolve().perform()
    obj_desig = DetectAction(milk_desig).resolve().perform()
     ParkArmsAction([Arms.BOTH]).resolve().perform()
     PickUpAction(spoon_desig, [Arms.LEFT], [Grasp.TOP]).resolve().perform()

     ParkArmsAction([Arms.BOTH]).resolve().perform()
     
```
</details>